In [202]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import scipy
import scipy.io
import matplotlib.pyplot as plt
import random
import pickle
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
from scipy.stats import pearsonr
from utils import *

In [203]:
device = torch.device(0)
device

device(type='cuda', index=0)

In [204]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1_train = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2_train = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3_train = np.delete(data_glove_3, 3, 1)

ecog_1_train = raw['train_ecog'][0][0]
ecog_2_train = raw['train_ecog'][1][0]
ecog_3_train = raw['train_ecog'][2][0]


raw = scipy.io.loadmat('./datasets/sub1_comp.mat')
ecog_1_comp = raw['train_data']
dg_1_comp = raw['train_dg']
ecog_1_valid = raw['test_data'][49000:]

raw = scipy.io.loadmat('./datasets/sub2_comp.mat')
ecog_2_comp = raw['train_data']
dg_2_comp = raw['train_dg']
ecog_2_valid = raw['test_data'][49000:]

raw = scipy.io.loadmat('./datasets/sub3_comp.mat')
ecog_3_comp = raw['train_data']
dg_3_comp = raw['train_dg']
ecog_3_valid = raw['test_data'][49000:]

dg_1_raw = scipy.io.loadmat('./datasets/sub1_testlabels.mat')
dg_1_valid = dg_1_raw['test_dg'][49000:]
dg_1_valid = np.delete(dg_1_valid, 3, 1)

dg_2_raw = scipy.io.loadmat('./datasets/sub2_testlabels.mat')
dg_2_valid = dg_2_raw['test_dg'][49000:]
dg_2_valid = np.delete(dg_2_valid, 3, 1)

dg_3_raw = scipy.io.loadmat('./datasets/sub3_testlabels.mat')
dg_3_valid = dg_3_raw['test_dg'][49000:]
dg_3_valid = np.delete(dg_3_valid, 3, 1)

In [214]:
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

In [206]:
class FingerFeatureDataset(Dataset):
    def __init__(self, R, dg, window=2000):
        self.R = np.float32(R + np.random.normal(0,0.1, R.shape))
#         self.R = (self.R - ecog_1_train.mean(axis=0)) / ecog_1_train.std(axis=0)
#         self.ecog = self.ecog.reshape(self.ecog.shape[0], 1, -1)
        self.dg = np.float32(dg)
        
    

    def __len__(self):
        return len(self.R)

    def __getitem__(self, idx):
        
        return self.R[idx], self.dg[idx]

In [216]:
def moving_average(x, winLen, winDisp):
    result = []
    for i in range(NumWins(x, 1000, winLen, winDisp)):
        result.append(x[i * int(winDisp * 1000):i * int(winDisp * 1000) + int(winLen * 1000)].mean(axis=0))
    return np.array(result)

In [279]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_1_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_1_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_1_train
Y_test = dg_1_valid
Y_train = moving_average(Y_train, winLen, winDisp)
Y_test = moving_average(Y_test, winLen, winDisp)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_1 = feature_selection(R_train, Y_train, 800)
print(idx_1.shape)

R_train = R_train[:, idx_1]
train_mean_S1 = R_train.mean(axis=0)
train_std_S1 = R_train.std(axis=0)
R_train = (R_train - train_mean_S1) / train_std_S1
R_test = R_test[:, idx_1]
R_test = (R_test - train_mean_S1) / train_std_S1

/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by z

(617,)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [287]:
train_mean_S1.shape

(617,)

In [281]:
dataset_s1_train = FingerFeatureDataset(R_train.copy(), Y_train.copy())
dataset_s1_valid = FingerFeatureDataset(R_test.copy(), Y_test.copy())

train_loader = DataLoader(dataset_s1_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s1_valid, batch_size=16, shuffle=False)

In [282]:
class FingerRegressor(nn.Module):
    def __init__(self, num_features, num_fingers) -> None:
        super(FingerRegressor, self).__init__()
        
        self.fc1 = nn.Linear(num_features, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, num_fingers)
        self.relu = nn.ReLU()
        
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)

        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.15)
        
    def forward(self, x):
        x = self.dropout1(x)
        x = self.relu(self.fc1(x))
        
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        output = self.fc3(x)

        
        return output

In [285]:
net = FingerRegressor(R_train.shape[1], 4).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())


for epoch in range(50):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_1_train)), data_glove_1_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(dg_1_valid)), dg_1_valid)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 1.503 | Train Cor: 0.099 | Valid loss: 0.926 | Valid Cor: 0.2675346607852328
Epoch 2 | Train loss: 0.881 | Train Cor: 0.234 | Valid loss: 0.835 | Valid Cor: 0.3458443107226765
Epoch 3 | Train loss: 0.801 | Train Cor: 0.296 | Valid loss: 0.810 | Valid Cor: 0.3755083202830025
Epoch 4 | Train loss: 0.752 | Train Cor: 0.337 | Valid loss: 0.789 | Valid Cor: 0.39684142499716446
Epoch 5 | Train loss: 0.719 | Train Cor: 0.373 | Valid loss: 0.769 | Valid Cor: 0.42615429974301683
Epoch 6 | Train loss: 0.707 | Train Cor: 0.394 | Valid loss: 0.768 | Valid Cor: 0.4404668158731721
Epoch 7 | Train loss: 0.680 | Train Cor: 0.417 | Valid loss: 0.759 | Valid Cor: 0.4542220340829953
Epoch 8 | Train loss: 0.660 | Train Cor: 0.444 | Valid loss: 0.752 | Valid Cor: 0.47623414055119656
Epoch 9 | Train loss: 0.654 | Train Cor: 0.452 | Valid loss: 0.752 | Valid Cor: 0.47671048577869957
Epoch 10 | Train loss: 0.626 | Train Cor: 0.478 | Valid loss: 0.746 | Valid Cor: 0.4921974244010766
Epoch

In [288]:
np.save('./models/train_mean_S1', train_mean_S1)
np.save('./models/train_std_S1', train_std_S1)
torch.save(net.state_dict(), './models/NN_S1.pth')

In [290]:
np.load('./models/train_mean_S1.npy').shape

(617,)

In [291]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_2_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_2_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_2_train
Y_test = dg_2_valid
Y_train = moving_average(Y_train, winLen, winDisp)
Y_test = moving_average(Y_test, winLen, winDisp)
# Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
# Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_2 = feature_selection(R_train, Y_train, 800)
print(idx_2.shape)

R_train = R_train[:, idx_2]
train_mean_S2 = R_train.mean(axis=0)
train_std_S2 = R_train.std(axis=0)
R_train = (R_train - train_mean_S2) / train_std_S2
R_test = R_test[:, idx_2]
R_test = (R_test - train_mean_S2) / train_std_S2

/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by z

(551,)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [292]:
dataset_s2_train = FingerFeatureDataset(R_train.copy(), Y_train.copy())
dataset_s2_valid = FingerFeatureDataset(R_test.copy(), Y_test.copy())

train_loader = DataLoader(dataset_s2_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s2_valid, batch_size=16, shuffle=False)

In [ ]:
net = FingerRegressor(R_train.shape[1], 4).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())


for epoch in range(50):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_2_train)), data_glove_2_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(dg_2_valid)), dg_2_valid)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 1.240 | Train Cor: 0.022 | Valid loss: 0.887 | Valid Cor: 0.12006371967436791
Epoch 2 | Train loss: 0.890 | Train Cor: 0.139 | Valid loss: 0.860 | Valid Cor: 0.1920142897589714
Epoch 3 | Train loss: 0.849 | Train Cor: 0.199 | Valid loss: 0.847 | Valid Cor: 0.25532671787720157
Epoch 4 | Train loss: 0.818 | Train Cor: 0.255 | Valid loss: 0.807 | Valid Cor: 0.3305694561753006
Epoch 5 | Train loss: 0.774 | Train Cor: 0.333 | Valid loss: 0.779 | Valid Cor: 0.37676357936190835
Epoch 6 | Train loss: 0.741 | Train Cor: 0.395 | Valid loss: 0.747 | Valid Cor: 0.42093487698794807
Epoch 7 | Train loss: 0.702 | Train Cor: 0.443 | Valid loss: 0.709 | Valid Cor: 0.45280104343304795
Epoch 8 | Train loss: 0.678 | Train Cor: 0.472 | Valid loss: 0.701 | Valid Cor: 0.4598256062524244
Epoch 9 | Train loss: 0.666 | Train Cor: 0.489 | Valid loss: 0.685 | Valid Cor: 0.47834449300979126
Epoch 10 | Train loss: 0.633 | Train Cor: 0.521 | Valid loss: 0.678 | Valid Cor: 0.4836669006163831
Epo

In [257]:
np.save('./models/train_mean_S2', train_mean_S2)
np.save('./models/train_std_S2', train_std_S2)
torch.save(net.state_dict(), './models/NN_S2.pth')

In [260]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_3_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_3_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_3_train
Y_test = dg_3_valid
Y_train = moving_average(Y_train, winLen, winDisp)
Y_test = moving_average(Y_test, winLen, winDisp)
# Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
# Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_3 = feature_selection(R_train, Y_train, 800)
print(idx_3.shape)

R_train = R_train[:, idx_3]
train_mean_S3 = R_train.mean(axis=0)
train_std_S3 = R_train.std(axis=0)
R_train = (R_train - train_mean_S3) / train_std_S3
R_test = R_test[:, idx_3]
R_test = (R_test - train_mean_S3) / train_std_S3

/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by z

(597,)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in divide
  correlation_coefficient /= X_norms
/opt/conda/envs/pytorch/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:358: RuntimeWarning: invalid value encountered in divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


In [269]:
dataset_s3_train = FingerFeatureDataset(R_train.copy(), Y_train.copy())
dataset_s3_valid = FingerFeatureDataset(R_test.copy(), Y_test.copy())

train_loader = DataLoader(dataset_s3_train, batch_size=8, shuffle=False)
test_loader = DataLoader(dataset_s3_valid, batch_size=8, shuffle=False)

In [270]:
# net = EEGNet().to(device)
# net = EEGNetRegressor(4).to(device)
# net = EEGNet(n_classes=4, channels=62, samples=3000).to(device) #.cuda(0)
net = FingerRegressor(R_train.shape[1], 4).to(device)
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))#.cuda(0))))
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())


for epoch in range(50):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog)
        pred += [output.detach().cpu().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = correlation_dl(sig.resample(pred, len(data_glove_3_train)), data_glove_3_train)[1]
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total
    
    net.eval()
    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = correlation_dl(sig.resample(pred, len(dg_3_valid)), dg_3_valid)[1]
        
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 1.382 | Train Cor: 0.167 | Valid loss: 0.732 | Valid Cor: 0.43330096698478415
Epoch 2 | Train loss: 0.762 | Train Cor: 0.382 | Valid loss: 0.641 | Valid Cor: 0.5442137449792854
Epoch 3 | Train loss: 0.653 | Train Cor: 0.499 | Valid loss: 0.585 | Valid Cor: 0.6128858844131725
Epoch 4 | Train loss: 0.600 | Train Cor: 0.550 | Valid loss: 0.554 | Valid Cor: 0.6398262418975164
Epoch 5 | Train loss: 0.573 | Train Cor: 0.587 | Valid loss: 0.532 | Valid Cor: 0.6561748603064894
Epoch 6 | Train loss: 0.542 | Train Cor: 0.617 | Valid loss: 0.511 | Valid Cor: 0.6810432826879983
Epoch 7 | Train loss: 0.510 | Train Cor: 0.641 | Valid loss: 0.494 | Valid Cor: 0.684175062093592
Epoch 8 | Train loss: 0.484 | Train Cor: 0.665 | Valid loss: 0.490 | Valid Cor: 0.6911981848142353
Epoch 9 | Train loss: 0.473 | Train Cor: 0.677 | Valid loss: 0.474 | Valid Cor: 0.7049498837771777
Epoch 10 | Train loss: 0.461 | Train Cor: 0.687 | Valid loss: 0.473 | Valid Cor: 0.7080266110873062
Epoch 11 

In [271]:
np.save('./models/train_mean_S3', train_mean_S3)
np.save('./models/train_std_S3', train_std_S3)
torch.save(net.state_dict(), './models/NN_S3.pth')

In [115]:
ecog_1_leaderboard = ecog_1_comp[500: 500 + 147500]
dg_1_leaderboard = dg_1_comp[500: 500 + 147500]

ecog_2_leaderboard = ecog_2_comp[500: 500 + 147500]
dg_2_leaderboard = dg_2_comp[500: 500 + 147500]

ecog_3_leaderboard = ecog_3_comp[500: 500 + 147500]
dg_3_leaderboard = dg_3_comp[500: 500 + 147500]

winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap


feature_1 = get_windowed_feats(ecog_1_leaderboard, 1000, winLen, winOverlap)
# R_1 = create_R_matrix(feature_1, 5)
feature_2 = get_windowed_feats(ecog_2_leaderboard, 1000, winLen, winOverlap)
# R_2 = create_R_matrix(feature_2, 5)
feature_3 = get_windowed_feats(ecog_3_leaderboard, 1000, winLen, winOverlap)
# R_3 = create_R_matrix(feature_3, 5)

# idx_1 = np.load('./models/idx_S1.npy')
# idx_2 = np.load('./models/idx_S2.npy')
# idx_3 = np.load('./models/idx_S3.npy')

R_1 = create_R_matrix(feature_1, 20)[:, idx_1]
R_2 = create_R_matrix(feature_2, 20)[:, idx_2]
R_3 = create_R_matrix(feature_3, 20)[:, idx_3]


/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '
/opt/conda/envs/pytorch/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '


In [116]:
train_mean_S1 = np.load('./models/train_mean_S1.npy')
train_std_S1 = np.load('./models/train_std_S1.npy')

train_mean_S2 = np.load('./models/train_mean_S2.npy')
train_std_S2 = np.load('./models/train_std_S2.npy')

train_mean_S3 = np.load('./models/train_mean_S3.npy')
train_std_S3 = np.load('./models/train_std_S3.npy')

In [117]:
leaderboard_dataset = FingerFeatureDataset((R_1 - train_mean_S1) / train_std_S1, np.zeros(R_1.shape[0]).copy())
dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

net = FingerRegressor(R_1.shape[1], 4).to(device)
net.load_state_dict(torch.load('./models/NN_S1.pth'))

with torch.no_grad():
    pred = []
    net.eval()
    for i, (ecog, dg) in enumerate(dataloader):
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog).to(device)
        pred += [output.detach().cpu().numpy()]

    pred = np.concatenate(pred)
correlation_dl(sig.resample(pred, len(dg_1_leaderboard)), dg_1_leaderboard)[1]

0.5553528686861728

In [118]:
leaderboard_dataset = FingerFeatureDataset((R_2 - train_mean_S2) / train_std_S2, np.zeros(R_2.shape[0]).copy())
dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

net = FingerRegressor(R_2.shape[1], 4).to(device)
net.load_state_dict(torch.load('./models/NN_S2.pth'))

with torch.no_grad():
    pred = []
    net.eval()
    for i, (ecog, dg) in enumerate(dataloader):
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog).to(device)
        pred += [output.detach().cpu().numpy()]

    pred = np.concatenate(pred)
correlation_dl(sig.resample(pred, len(dg_2_leaderboard)), dg_2_leaderboard)[1]

0.5654337122490332

In [119]:
leaderboard_dataset = FingerFeatureDataset((R_3 - train_mean_S3) / train_std_S3, np.zeros(R_3.shape[0]).copy())
dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

net = FingerRegressor(R_3.shape[1], 4).to(device)
net.load_state_dict(torch.load('./models/NN_S3.pth'))

with torch.no_grad():
    pred = []
    net.eval()
    for i, (ecog, dg) in enumerate(dataloader):
        ecog = ecog.to(device)
        dg = dg.to(device)
        output = net(ecog).to(device)
        pred += [output.detach().cpu().numpy()]

    pred = np.concatenate(pred)
correlation_dl(sig.resample(pred, len(dg_3_leaderboard)), dg_3_leaderboard)[1]

0.6924273174017038